<a href="https://colab.research.google.com/github/Akshara-Balan/Parallel-NE-Pair-and-Sentence-Generation--Bhashathon/blob/main/Draft_code_that_works_for_small_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

aksharabalan_bpcc_data_path = kagglehub.dataset_download('aksharabalan/bpcc-data')
aksharabalan_test_short_path = kagglehub.dataset_download('aksharabalan/test-short')

print('Data source import complete.')


In [ ]:
!pip install stanza deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import stanza
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load dataset
def load_dataset(file_path):
    """ Load English-Malayalam dataset """
    return pd.read_csv(file_path)

# Load English NER model (Stanza)
stanza.download("en")
nlp_en = stanza.Pipeline(lang="en", processors="tokenize,ner")

# Load fine-tuned Malayalam NER model
model_name = "AksharaBalan/malayalam-bert-finetuned-ner"
tokenizer_ml = AutoTokenizer.from_pretrained(model_name)
model_ml = AutoModelForTokenClassification.from_pretrained(model_name)

# Function to extract NEs using Stanza (English)
def extract_ner_stanza(text):
    doc = nlp_en(text)
    return [ent.text for ent in doc.ents]

# Function to extract NEs using Malayalam NER model
def extract_ner_malayalam(text):
    tokens = tokenizer_ml(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model_ml(**tokens).logits
    predictions = torch.argmax(outputs, dim=-1).tolist()[0]
    return [tokenizer_ml.decode(tokens["input_ids"][0][i]) for i in range(len(predictions)) if predictions[i] > 0]  # Exclude non-entities

# Process dataset
def process_dataset(file_path):
    df = load_dataset(file_path)

    aligned_ner_pairs = []

    for _, row in df.iterrows():
        en_text = row['src']
        ml_text = row['tgt']

        en_entities = extract_ner_stanza(en_text)
        ml_entities = extract_ner_malayalam(ml_text)

        aligned_ner_pairs.append({"English": en_entities, "Malayalam": ml_entities})

    df["aligned_ner_pairs"] = aligned_ner_pairs

    # Save the refined dataset
    df.to_csv("aligned_parallel_corpus.csv", index=False)
    print("Processed dataset saved as 'aligned_parallel_corpus.csv'.")

# Run the processing
file_path = "/kaggle/input/test-short/test.csv"  # Update with actual file path
process_dataset(file_path)


Processed dataset saved as 'aligned_parallel_corpus.csv'.


In [ ]:
import pandas as pd

# Load aligned dataset
df = pd.read_csv("aligned_parallel_corpus.csv")

# Function to check NE mismatches and fix them
def fix_named_entity_mismatch(row):
    en_sentence = row["src"]
    ml_sentence = row["tgt"]
    en_entities = eval(row["aligned_ner_pairs"])["English"]  # Convert back to list
    ml_entities = eval(row["aligned_ner_pairs"])["Malayalam"]

    corrected_ml_sentence = ml_sentence

    # Check for missing or incorrect NEs
    for i, en_ne in enumerate(en_entities):
        if i < len(ml_entities):
            ml_ne = ml_entities[i]
            if ml_ne not in ml_sentence:
                corrected_ml_sentence = corrected_ml_sentence.replace(ml_ne, en_ne)  # Replace incorrect NE

    return corrected_ml_sentence

# Apply correction
df["corrected_tgt"] = df.apply(fix_named_entity_mismatch, axis=1)

# Save corrected corpus
df.to_csv("corrected_parallel_corpus.csv", index=False)
print("Corrected parallel corpus saved as 'corrected_parallel_corpus.csv'.")


Corrected parallel corpus saved as 'corrected_parallel_corpus.csv'.


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
from sacrebleu import corpus_bleu

# Load original and corrected target sentences
original_refs = df["tgt"].tolist()
corrected_preds = df["corrected_tgt"].tolist()

# Compute BLEU Score
bleu_score = corpus_bleu(corrected_preds, [original_refs])
print(f"BLEU Score for corrected corpus: {bleu_score.score}")


BLEU Score for corrected corpus: 100.00000000000004


In [ ]:
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import torch

# Load LaBSE model
labse_model_name = "sentence-transformers/LaBSE"
tokenizer_labse = AutoTokenizer.from_pretrained(labse_model_name)
model_labse = AutoModel.from_pretrained(labse_model_name)

def get_embedding(text):
    """Generate sentence embedding using LaBSE."""
    tokens = tokenizer_labse(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model_labse(**tokens)
    return outputs.pooler_output[0].numpy()

# Compute similarity scores
df["labse_similarity"] = df.apply(lambda row: 1 - cosine(get_embedding(row["src"]), get_embedding(row["corrected_tgt"])), axis=1)


In [ ]:
# Filter sentence pairs with similarity threshold (e.g., 0.75)
df_filtered = df[df["labse_similarity"] >= 0.75]

# Save final dataset
df_filtered.to_csv("final_parallel_corpus.csv", index=False)
print(f"Filtered dataset saved with {len(df_filtered)} high-quality sentence pairs!")


Filtered dataset saved with 24 high-quality sentence pairs!
